#### Simple Gen AI APP Using Langchain

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
# Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [15]:
## Data Ingestion == from the. website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/self_hosting/installation/docker")
loader


In [16]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/self_hosting/installation/docker', 'title': 'Self-hosting LangSmith with Docker | 🦜️🛠️ LangSmith', 'description': 'Self-hosting LangSmith is an add-on to the Enterprise Plan designed for our largest, most security-conscious customers. See our pricing page for more detail, and contact us at sales@langchain.dev if you want to get a license key to trial LangSmith in your environment.', 'language': 'en'}, page_content='\n\n\n\n\nSelf-hosting LangSmith with Docker | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingArchitectural overviewScriptsInstallationInstalling on KubernetesInstalling on DockerConfigurationUsageUpgradesEgress for Subscription Metrics and Operational MetadataO

In [17]:
### Load Data  --> Get all the docs --> Divide out text into chunks --> Convert this into vector by using Vector Embeddings -> store in Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=30)
documents = text_splitter.split_documents(docs)
print(documents)

[Document(metadata={'source': 'https://docs.smith.langchain.com/self_hosting/installation/docker', 'title': 'Self-hosting LangSmith with Docker | 🦜️🛠️ LangSmith', 'description': 'Self-hosting LangSmith is an add-on to the Enterprise Plan designed for our largest, most security-conscious customers. See our pricing page for more detail, and contact us at sales@langchain.dev if you want to get a license key to trial LangSmith in your environment.', 'language': 'en'}, page_content='Self-hosting LangSmith with Docker | 🦜️🛠️ LangSmith'), Document(metadata={'source': 'https://docs.smith.langchain.com/self_hosting/installation/docker', 'title': 'Self-hosting LangSmith with Docker | 🦜️🛠️ LangSmith', 'description': 'Self-hosting LangSmith is an add-on to the Enterprise Plan designed for our largest, most security-conscious customers. See our pricing page for more detail, and contact us at sales@langchain.dev if you want to get a license key to trial LangSmith in your environment.', 'language': '

In [18]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/self_hosting/installation/docker', 'title': 'Self-hosting LangSmith with Docker | 🦜️🛠️ LangSmith', 'description': 'Self-hosting LangSmith is an add-on to the Enterprise Plan designed for our largest, most security-conscious customers. See our pricing page for more detail, and contact us at sales@langchain.dev if you want to get a license key to trial LangSmith in your environment.', 'language': 'en'}, page_content='Self-hosting LangSmith with Docker | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/self_hosting/installation/docker', 'title': 'Self-hosting LangSmith with Docker | 🦜️🛠️ LangSmith', 'description': 'Self-hosting LangSmith is an add-on to the Enterprise Plan designed for our largest, most security-conscious customers. See our pricing page for more detail, and contact us at sales@langchain.dev if you want to get a license key to trial LangSmith in your environment.', 'language': 

In [19]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [20]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [21]:
vectorstoredb

In [ ]:
## Query from a vector DB
query = "What to do if we don't see any server information in the output?"
result = vectorstoredb.similarity_search(query)
result[0].page_content

"docker info\nIf you don't see any server information in the output, make sure Docker is installed correctly and launch the Docker daemon."

In [25]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [29]:
## Retrieval Chain, Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x11b238830>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x11b23b230>, root_client=<openai.OpenAI object at 0x11b620550>, root_async_client=<openai.AsyncOpenAI object at 0x11b620b90>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume

In [30]:
from langchain_core.documents import Document
document_chain.invoke({"input":"What to do if we don't see any server information in the output?",
                       "context":[Document(page_content="If you don't see any server information in the output, make sure Docker is installed correctly and launch the Docker daemon. And consult with Aakash kumar jha fr further Help")]})

"If you don't see any server information in the output, ensure Docker is installed properly, launch the Docker daemon, and consult Aakash Kumar Jha for further help."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [31]:
## Input --->>>Retriever-  it can be consider as an interface, and it responsibility is ---> VectorstoreDb
## so retrievr is a kinf of an interface for the vector db
vectorstoredb
 

In [33]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever , document_chain)

In [34]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1182f0910>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [ ]:
response = retriever_chain.invoke({"input":"What to do if we don't see any server information in the output?"})


In [37]:
response['answer']

"The provided context includes instructions and information related to Docker and server management. Here are some key points and guidance based on the context:\n\n1. **Docker Installation Check**: If you run the command `docker info` and do not see server information in the output, it's advisable to verify that Docker is installed correctly and ensure the Docker daemon is running.\n\n2. **Viewing Server Logs**: To check if the server is running and to view its logs, you can use the command `docker-compose logs`.\n\n3. **Stopping the Server**: To stop the server, execute the command `docker-compose down`.\n\nThe information also references some configuration details related to a service at `localhost:1980` with a version specified as `0.5.7` and a license expiration date of `2033-05-20T20:08:06`, but without more specific context, it is not clear what this service pertains to."